# Lists of S&P 500 companies historical components

1. Read 'S&P 500 Historical Components & Changes.csv' which contains historical S&P 500 index membership since 1996.  Each row contains 2 fields: date and tickers; tickers is a comma delimited string of the symbols for that specified date.  This file was provided as open source via download for the book "Trading Evolved" by Andreas F. Clenow.  
2. Clean up the list by fixing up symbols, removing duplicates in each row, and sorting the symbols  
3. Read in 'sp500_changes_since_2019.csv' which contains the changes to the index since 2019  
4. Create new rows in the historical membership list for these changes  
5. Compare the last row to current list of S&P 500 components on the wikipedia  
6. A list of differences is generated.  If this list is NOT empty, then changes have been made to the index that should be included in 'sp500_changes_since_2019.csv'.  
7. Write out date stamped 'S&P 500 Historical Components & Changes' csv file

In [1]:
# other imports
from datetime import datetime
import pandas as pd
import os

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 600)

# -*- encoding: utf-8 -*-
%matplotlib inline

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
def get_table(filename):

    if os.path.isfile(filename):
        df = pd.read_csv(filename, index_col='date')
        return df

In [4]:
filename = 'S&P 500 Historical Components & Changes.csv'
df = get_table(filename)
df.head()

,tickers
date,
1996-01-02,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,ABI-20..."
1996-01-03,"AAL-199702,AAMRQ-201312,AAPL,ABI-200811,ABS-20..."
1996-01-04,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,OAT-20..."
1996-01-10,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,ABI-20..."
1996-01-11,"FLMIQ-200408,FLTWQ-200907,FMC,FMCC,FNMA,FTL.A-..."


In [5]:
# convert ticker column from csv to list, then sort
df['tickers'] = df['tickers'].apply(lambda x: sorted(x.split(',')))
df.tail()

,tickers
date,
2019-01-03,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-08,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-09,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-10,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-11,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


In [6]:
# replace SYMBOL-yyyymm with SYMBOL
df['tickers'] = [[ticker.split('-')[0] for ticker in tickers] for tickers in df['tickers']]
df.head()

,tickers
date,
1996-01-02,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-03,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-04,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-10,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-11,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."


In [7]:
# add LIN after 2018-10-31
def _add_symbol(row, symbol, date):
    if row.name > date:
        row.tickers.append(symbol)
    return row.tickers
df['tickers'] = df.apply(_add_symbol, symbol='LIN', date='2018-10-31', axis=1)

In [8]:
# remove duplicates in each row
df['tickers'] = [sorted(list(set(tickers))) for tickers in df['tickers']]
df.tail()

,tickers
date,
2019-01-03,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-08,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-09,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-10,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-11,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


Changes to the list of S&P 500 components
https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components

In [9]:
# read S&P 500 changes csv file
filename = 'sp500_changes_since_2019.csv'
changes = get_table(filename)

# convert ticker column from csv to list, then sort
changes['add'] = changes['add'].apply(lambda x: sorted(x.split(',')))
changes['remove'] = changes['remove'].apply(lambda x: sorted(x.split(',')))
changes

,add,remove
date,,
2019-01-18,[TFX],[PCG]
2019-02-15,[ATO],[NFX]
2019-02-27,[WAB],[GT]
2019-04-02,[DOW],[BHF]
2019-06-01,[LHX],[HRS]
2019-06-03,"[CTVA, DD]","[DWDP, FLR]"
2019-06-07,[AMCR],[MAT]
2019-07-01,[MKTX],[LLL]
2019-07-15,[TMUS],[RHT]


In [10]:
# copy the last row in dataframe, modify for changes, then append
for change in changes.itertuples():

    new_row = df.tail(1)
    
    tickers = list(new_row['tickers'][0])
    tickers += change.add
    tickers = list(set(tickers) - set(change.remove))
    tickers = sorted(tickers)
    
    d = {'date':change.Index, 'tickers':[tickers]}
    new_entry = pd.DataFrame(d)
    new_entry.set_index('date', inplace=True)
    df = df.append(new_entry)

In [11]:
df.tail()

,tickers
date,
2020-10-07,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2020-10-12,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2020-11-17,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2020-12-21,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2021-01-07,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


In [12]:
# compare last row to current S&P500 list
filename = 'sp500.csv'
current = pd.read_csv(filename)
current = list(current['Symbol'])
last_entry = list(df['tickers'][-1])

diff = list(set(current) - set(last_entry)) + list(set(last_entry) - set(current))
diff

[]

In [13]:
# convert tickers column back to csv
df['tickers'] = df['tickers'].apply(lambda x: ",".join(x))
df.head()

,tickers
date,
1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


In [14]:
now = datetime.now()
dt_string = now.strftime('%m-%d-%Y') # mm-dd-YYYY
filename = 'S&P 500 Historical Components & Changes({}).csv'.format(dt_string)
df.to_csv(filename)